<a href="https://colab.research.google.com/github/limpa105/RegexPlus/blob/neural_stuff/neural/neural_speaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install hypothesis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import string 
import random 
from numpy.random import choice
from scipy.stats import skewnorm

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import hypothesis

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
def generate_training_data(num_examples: int, max_size:int, regex_things:list) -> list:
  examples = [ [ regex_things[random.randint(0,len(regex_things)-1)] for  i in range(random.randint(1,max_size))] for k in range(num_examples)]
  return examples
  
def generate_training_data_pairs(training_data: list) -> list:
  return [(i, hypothesis.strategies.from_regex('^' + ''.join(i) + '$').example().strip()) for i in training_data]

In [39]:
# make the language 
# first iteration = very restricted language no optionals no constants
regex_things : list = ['[0-9]','[a-z]','[A-Z]','[a-zA-Z]', '[a-zA-Z0-9]', '[0-9]+','[a-z]+','[A-Z]+','[a-zA-Z]+', '[a-zA-Z0-9]+']
ascii_char : list = list(string.printable)[:95]
special_char = ['.', '\\', '+', '*', ')', '(', '?' ]
#regex_things = regex_things + [ i for i in ascii_char if i not in special_char]
regex_things = regex_things + ascii_char[:62]
# takes some time to run 
# TODO: Write this into a file and read from a file
training_data = generate_training_data(3000, 10, regex_things)
pairs = generate_training_data_pairs(training_data)

In [63]:
pairs

[(['6', 'r', 'p', '7', 'G', '[0-9]', '[a-z]+', 'i', 'r'], '6rp7G2psjoepzir'),
 (['l'], 'l'),
 (['M', 'W', '[A-Z]', 'l', 'n', '4', 'H', 'T', 'D'], 'MWJln4HTD'),
 (['R', '1', 'u'], 'R1u'),
 (['r', 'W', '2', 'B', '[a-zA-Z0-9]', 'U', 'S', 's', 'M', 'h'], 'rW2BAUSsMh'),
 (['J', 'X', 'a', 'l'], 'JXal'),
 (['c', 'y', '[0-9]'], 'cy8'),
 (['5', '8'], '58'),
 (['[0-9]+', 'r', 'A', 'V'], '7rAV'),
 (['e', 'l', 'T', 'G', 'c', 'n', 'z', 'Z', 'd', 'b'], 'elTGcnzZdb'),
 (['b', 'l', 'z', '8', 'F', 'S', '7'], 'blz8FS7'),
 (['t', '[a-zA-Z]+', 'E'], 'tiEEEE'),
 (['t',
   'e',
   '[A-Z]',
   '2',
   '[a-zA-Z]+',
   'w',
   '[a-zA-Z0-9]',
   '[0-9]+',
   'e',
   '[a-z]'],
  'teI2Kw728853604712663ep'),
 (['G', 'V', '8'], 'GV8'),
 (['b', 'a', 'i'], 'bai'),
 (['z', 'r', '[0-9]'], 'zr7'),
 (['u', 'S', 'P', 'E', 'c', 'r', 'I'], 'uSPEcrI'),
 (['N', 'o'], 'No'),
 (['y'], 'y'),
 (['l', '[a-z]+', 'B', 'A', 'y'], 'lbjzxxyBAy'),
 (['O', 'I', 'A', 'T', 'T', 'C'], 'OIATTC'),
 (['T', 'd', '8', 'j', 'H', 'A'], 'Td8jHA'),


In [42]:
SOS_token = 0
EOS_token = 1

# constructing langiage using 
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
      
    def addList(self, list_words):
        for word in list_words:
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [43]:
MAX_LENGTH = 12
def readLangs(lang1: list, lang2:list, reverse=False):
  input_lang = Lang('regex')
  output_lang = Lang('english')
  input_lang.addList(lang1)
  output_lang.addList(lang2)
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  return input_lang, output_lang


In [44]:
input_lang, output_lang = readLangs(regex_things, ascii_char)

regex 74
english 97


In [45]:
# model code also taken from 
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [46]:
def indexesFromList(lang, char_list):
    return [lang.word2index[word] for word in char_list]


def tensorFromList(lang, char_list):
    indexes = indexesFromList(lang, char_list)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromList(input_lang, pair[0])
    target_tensor = tensorFromList(output_lang, list(pair[1]))
    return (input_tensor, target_tensor)


In [47]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [48]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [49]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [50]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [51]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromList(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [52]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 40000, print_every=1000)

0m 13s (- 8m 46s) (1000 2%) 2.6895
0m 25s (- 7m 55s) (2000 5%) 2.7149
0m 37s (- 7m 46s) (3000 7%) 2.7872
0m 51s (- 7m 46s) (4000 10%) 2.5168
1m 5s (- 7m 40s) (5000 12%) 2.2577
1m 19s (- 7m 32s) (6000 15%) 2.0636
1m 34s (- 7m 24s) (7000 17%) 1.9498
1m 48s (- 7m 15s) (8000 20%) 1.7780
2m 3s (- 7m 5s) (9000 22%) 1.6776
2m 18s (- 6m 54s) (10000 25%) 1.5506
2m 32s (- 6m 43s) (11000 27%) 1.3484
2m 47s (- 6m 31s) (12000 30%) 1.3656
3m 2s (- 6m 19s) (13000 32%) 1.3526
3m 17s (- 6m 6s) (14000 35%) 1.3222
3m 31s (- 5m 53s) (15000 37%) 1.2771
3m 46s (- 5m 39s) (16000 40%) 1.2643
4m 2s (- 5m 27s) (17000 42%) 1.4874
4m 16s (- 5m 13s) (18000 45%) 1.6140
4m 30s (- 4m 59s) (19000 47%) 1.3280
4m 45s (- 4m 45s) (20000 50%) 1.2491
4m 59s (- 4m 31s) (21000 52%) 1.2106
5m 14s (- 4m 17s) (22000 55%) 1.4881
5m 29s (- 4m 3s) (23000 57%) 1.4011
5m 43s (- 3m 49s) (24000 60%) 3.1052
5m 57s (- 3m 34s) (25000 62%) 3.9434
6m 11s (- 3m 20s) (26000 65%) 4.1394
6m 26s (- 3m 5s) (27000 67%) 4.1213
6m 40s (- 2m 51s) (28

In [64]:
evaluate(encoder1, attn_decoder1, ['[0-9]', '[a-z]', '[A-Z]'])

(['0', '<EOS>'], tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))